# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [1]:
!pip install -qU openai

In [2]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

## Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [3]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [5]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "Pebble" # @param {type: "string"}
instructions = "You are great in coming up with ideas for garden landscaping." # @param {type: "string"}
model = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [6]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [7]:
assistant

Assistant(id='asst_jPr7PjPHDDak55gdmkOUalFz', created_at=1713244157, description=None, file_ids=[], instructions='You are great in coming up with ideas for garden landscaping.', metadata={}, model='gpt-3.5-turbo', name='Pebble', object='assistant', tools=[], top_p=1.0, temperature=1.0, response_format='auto')

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [8]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [9]:
thread

Thread(id='thread_JRRyAFaPcJUFBUqLpmupzvsi', created_at=1713244210, metadata={}, object='thread')

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [10]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"Pebble, help me layout a vegetable garden plan that maximizes space."
)

Again, let's examine our `message` object!

In [11]:
message

Message(id='msg_l4dc8MDQPFPj64wczOOVizmm', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Pebble, help me layout a vegetable garden plan that maximizes space.'), type='text')], created_at=1713244432, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_JRRyAFaPcJUFBUqLpmupzvsi')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [12]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = "think step by step" # @param {type: "string"}

Let's run our Thread!

In [13]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=instructions + " " + additional_instructions
)

Now that we've run our thread, let's look at the object!

In [14]:
run

Run(id='run_Kx2LdMzAp4Jqeoq7l0KP2a1y', assistant_id='asst_jPr7PjPHDDak55gdmkOUalFz', cancelled_at=None, completed_at=None, created_at=1713244549, expires_at=1713245149, failed_at=None, file_ids=[], incomplete_details=None, instructions='You are great in coming up with ideas for garden landscaping. think step by step', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_JRRyAFaPcJUFBUqLpmupzvsi', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0)

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [15]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [16]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [17]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [18]:
messages.data[0]

Message(id='msg_Hayhtc4gxTzI7Fcui3CszDJz', assistant_id='asst_jPr7PjPHDDak55gdmkOUalFz', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Sure! Here is a step-by-step guide to help you create a vegetable garden plan that maximizes space using pebbles as a decorative element:\n\n1. Measure the available space: Start by measuring the area where you want to create your vegetable garden. Note any obstacles such as trees, fences, or other structures that may impact the layout.\n\n2. Design your garden layout: Sketch out a rough design of your vegetable garden layout. Consider using raised beds or containers to maximize space and make it easier to maintain. Divide the space into sections for different types of vegetables, herbs, and flowers.\n\n3. Create pathways: Use pebbles to create defined pathways between the different sections of your vegetable garden. This will not only help with navigation but also add a decorative element to your garden.\n\n4. Use pebble

## Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Creating an Assistant with the Retriever Tool

The first thing we'll want to do is create an assistant with the Retriever tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data

First, we need some data. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [19]:
!wget https://www.gutenberg.org/files/84/84-h/84-h.htm -o frankenstein.html

Now we can upload our file!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [20]:
file_reference = client.files.create(
  file=open("frankenstein.html", "rb"),
  purpose='assistants'
)

Let's look at what our `file_reference` contains!

In [57]:
file_reference

FileObject(id='file-UOUQLfLNOKThQNRYGKEvsIvx', bytes=1200, created_at=1713246476, filename='frankenstein.html', object='file', purpose='assistants', status='processed', status_details=None)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Please pay attention to [pricing](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) and don't forget to delete your files when you're done!

In [58]:
assistant = client.beta.assistants.create(
  name=name + "+ Retrieval",
  instructions=instructions,
  model=model,
  tools=[{"type": "retrieval"}],
  file_ids=[file_reference.id]
)

Let's try submitting a message to our Assistant and seeing what kind of answer we get!

We'll outline the steps needed to do this in full:

1. Create an Assistant
2. Create a Thread
3. Add Messages to that Thread
4. Create a Run on that Thread
5. Wait for Run to Complete
6. Collect Messages from the Thread

Let's do that below!

In [59]:
# Create a Thread
thread = client.beta.threads.create()

# Add Messages to that Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What is the first words Victor Frankenstein speaks?"
)

# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


Let's look at the final result!

In [60]:
messages

SyncCursorPage[Message](data=[Message(id='msg_upsOc3lXz8Y2ea90wxclt3DR', assistant_id='asst_EOZtFQxWpWmPUY6CVrsiFZNe', completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=226, file_citation=FileCitation(file_id='file-UOUQLfLNOKThQNRYGKEvsIvx', quote=''), start_index=213, text='【19:0†source】', type='file_citation')], value='The first words spoken by Victor Frankenstein are: "You seek for knowledge and wisdom, as I once did; and I ardently hope that the gratification of your wishes may not be a serpent to sting you, as mine has been."【19:0†source】.'), type='text')], created_at=1713279591, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_xHZuLN5mXzOpZhWGTc3hOCAC', status=None, thread_id='thread_w1TumrsrBjFPs4LAtPqd19bl'), Message(id='msg_GidFDXY1MK5kSfVr2Fq48lMO', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What 

Let's do some clean up to make sure we're not being charged anything extra by deleting our resources.

In [61]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)

### Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [62]:
assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [63]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "file_ids": [file_reference.id]
    }
  ]
)

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [64]:
# Create a Run on that Thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Wait for Run to Complete
while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

# Collect Messages from the Thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

queued
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress


We can check the specific steps that the Code Interpreter ran to figure out what steps the Assistant took!

In [65]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [66]:
for step in run_steps.data:
  print(step.step_details)

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_DYb4C4GwyujyrAObnCnj0etb'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_h869SMSIJaNVPpfRj5MaGsbU', code_interpreter=CodeInterpreter(input="# Function to read the first few bytes of the file\r\ndef read_file_header(file_path, num_bytes=16):\r\n    with open(file_path, 'rb') as file:\r\n        header = file.read(num_bytes)\r\n    return header\r\n\r\n# Read the first 16 bytes of the file\r\nfile_header = read_file_header(file_path, num_bytes=16)\r\nfile_header", outputs=[CodeInterpreterOutputLogs(logs="b'--2024-04-16 01:'", type='logs')]), type='code_interpreter')], type='tool_calls')
MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_xMMCLz9IkVsPfTzrVjhQ9405'), type='message_creation')
ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_EYACp1p04bAcBeGI2Z5rssIU', code_interpreter=CodeInterpreter(input="import os\r\n\r\n# Get t

In [67]:
messages

SyncCursorPage[Message](data=[Message(id='msg_DYb4C4GwyujyrAObnCnj0etb', assistant_id='asst_VyYOGAAjBAbQM7h1u06vYmKk', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="The first few bytes of the file are `--2024-04-16 01:`, which doesn't give a clear indication of the file type. At this point, it's not immediately apparent what kind of file this is based on the header. \n\nIf you have any information about the file type or its expected contents, please let me know. Otherwise, we can try opening the file with different applications or examining its contents in different ways to determine its type. Let me know how you would like to proceed."), type='text')], created_at=1713279666, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_mwGyCSlofqWrmEurxiwK2qKz', status=None, thread_id='thread_6HhdcQxWPp56vO8x1xQ4GpqS'), Message(id='msg_xMMCLz9IkVsPfTzrVjhQ9405', assistant_id='asst_VyYOGAAjB

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [68]:
!pip install -qU duckduckgo_search

In [69]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [70]:
duckduckgo_search("Who is the current captain of the Winnipeg Jets?")

"Lowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ...\nAdam Lowry, who has been a Jet since 2011 when he was drafted 67th overall, is the new captain of the NHL team — its third since relocating to Winnipeg from Atlanta in 2011. Andrew Ladd served ...\nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ...\nWinnipeg played without a captain last season after stripping Blake Wheeler of the title in September. Lowry is entering his 10th season with the Jets, who drafted him in the third round in 2011.\nThe Winnipeg Jets have officially made the decision on which player will wear the captain's 'C' for the 2023-24 season, and hopefully onward. That player is 30-year-old centre Adam Lowry."

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [71]:
ddg_function = {
    "name" : "duckduckgo_search",
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
    }
}

####❓ Question

Why does the description key-value pair matter?

Answer - It is used by the LLM to pick the appropriate tool based on the prompt it receives.

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [72]:
assistant = client.beta.assistants.create(
    name=name + " + Function Calling API",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

We need to make a few modifications to our Assistant to include the ability to make calls to our local function and pass the results back to our Assistant for further generation.

In [73]:
import json

def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "duckduckgo_search":
            print("Consulting Duck Duck Go...")
            output = duckduckgo_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    print(tool_output_array)

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    for msg in messages:
        print(f"{msg.role}: {msg.content[0].text.value}")

def use_assistant(query, assistant_id, thread_id=None):
  thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content=query,
  )

  print("Creating Assistant ")

  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  print("Querying OpenAI Assistant Thread.")

  run = wait_for_run_completion(thread.id, run.id)

  if run.status == 'requires_action':
    run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
    run = wait_for_run_completion(thread.id, run.id)

  print_messages_from_thread(thread.id)

  return thread.id

####❓ Question

Outline, in simple terms, what the `use_assistant` helper function is doing.

Answer - The function initiates a conversation with the assistant by creating a thread and logging the user's input as a message in that thread. It then executes the thread and monitors it until completion. If the execution status indicates that a tool is required, the function triggers the corresponding tool, produces the necessary output, and links this output to a new run using the `submit_tools_output` method. The function then waits for this new run to finish and subsequently retrieves the messages from the completed run.

In [74]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_29IUu26e4ZFuOXnuGH7bo4Az', 'output': 'Lowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ...\nWinnipeg Jets Captains. Team Names: Winnipeg Jets, Atlanta Thrashers. Seasons: 24 (1999-00 to 2023-24) NHL Playoff Appearances: 7. NHL Championships: 0 (0 Stanley Cup) ... Current Summary/Standings, Current Schedule/Results, Current Leaders, Current Stats. 2023-24, ...\nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ...\nThe Winnipeg Jets have a new leader, one year after stripping the C from Blake Wheeler and deciding to play without a 

'thread_UmID6J7X1BOxP2rHjBeJQWi8'

## Wrapping it All Together

Now we can create an Assistant with all of the available tools and see how it responds to various queries!

In [75]:
assistant = client.beta.assistants.create(
    name=name + " + All Tools",
    instructions=instructions,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function" : ddg_function}
    ],
    model=model,
    file_ids=[file_reference.id],
)

In [76]:
use_assistant("Who is the current Captain of the Winnipeg Jets?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: I couldn't find information about the current Captain of the Winnipeg Jets in the document you uploaded. Would you like me to look up this information online for you?
user: Who is the current Captain of the Winnipeg Jets?


'thread_TPlvHgusOalsqS3coGziUqnJ'

In [77]:
use_assistant("Who is the author of the supplied file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: The author of the supplied file is Mary Shelley, and the file appears to be the text of the novel "Frankenstein" by Mary Shelley.
user: Who is the author of the supplied file?


'thread_C7DcQLPhaO83MGNfrmaHfnG6'

In [78]:
use_assistant("How many bytes is the provided file?", assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_at39cF5e3f1VxmuX2sHlG2I1', 'output': 'Here are some advantages to use du command to check file size in Linux. The du command can show the size of a file in human-readable format, which makes it easier to interpret the results.; The du command can be used to check the size of multiple files at once.; The du command can be used to check the size of directory.; The du command is faster than the ls command when checking the size of ...\nThe du command can be used to check the size of files, directories, and the total disk space used by the current directory and subdirectories. Run "du -h" to see a list of files and folders in a human-readable format. When you use the Linux du command, you obtain both the actual disk usage and the true size of a file or directory.\nBy default, the block size in most Linux system is 4096 Bytes or 4 KB. A directory in Li

'thread_AM26tPWPjiY48oTL1C5i0OyA'

####❓ Question

Notice that our response can go through multiple paths, given that:

What is "deciding" to use the tool?

Answer - The LLM model associated to the assistant helps to identify the path and tool the assistant to levearage to answer the prompt.

### Adding JSON Mode for More Agentic Behaviour

Finally, we have the ability to select tools - all we need to do now is set up a process to allow us to create some kind of loop and make decisions about whether or not the response is complete or not.

We'll leverage the OpenAI completions end-endpoint with JSON mode to let us understand when we've adequately answered our user's question!

In [79]:
completed_template = \
"""
Does this response adequately answer the user's query?

Please return your response in JSON format - with key: "completed" and either True (if completed) or False (if not completed)

User Query:
{query}

Assistant Response:
{response}
"""

def is_complete(query, response):
  completed_response = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": completed_template.format(query=query, response=response),
          }
      ],
      model=model,
      response_format={"type" : "json_object"}
  )

  return completed_response

In [80]:
query = "How many bytes is the provided file?"

thread_id_for_response = use_assistant(query, assistant.id)

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Current run status: completed
assistant: The provided file is 1200 bytes in size.
user: How many bytes is the provided file?


Now we can observe JSON mode in action!

In [81]:
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
response = messages.data[0].content[0].text.value
completed_flag = json.loads(is_complete(query, response).choices[0].message.content)

In [82]:
completed_flag

{'completed': True}

## 🚧 BONUS CHALLENGE 🚧:

Use the components we've constructed so far to build a loop that lets us continue to query the Assistant if the response is not completed!

In [92]:
### YOUR CODE HERE
query = "What are the health benefits of daily meditation?"
thread_id_for_response = use_assistant(query, assistant.id)
messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
response = messages.data[0].content[0].text.value
completed_flag = json.loads(is_complete(query, response).choices[0].message.content)

num_iterations = 2
while (not completed_flag["completed"]) and num_iterations < 5:
    print("loop number---- " + str(num_iterations))
    #use the same thread and re-run
    run = client.beta.threads.runs.create(
        thread_id=thread_id_for_response,
        assistant_id=assistant.id,
    )

    run = wait_for_run_completion(thread_id_for_response, run.id)

    if run.status == 'requires_action':
        run = submit_tool_outputs(thread_id_for_response, run.id, run.required_action.submit_tool_outputs.tool_calls)
        run = wait_for_run_completion(thread_id_for_response, run.id)
    
    messages = client.beta.threads.messages.list(thread_id=thread_id_for_response)
    response = messages.data[0].content[0].text.value
    completed_flag = json.loads(is_complete(query, response).choices[0].message.content)
    num_iterations +=  1

completed_flag

Creating Assistant 
Querying OpenAI Assistant Thread.
Current run status: in_progress
Current run status: completed
assistant: I couldn't find specific information about the health benefits of daily meditation in the uploaded file. Would you like me to conduct a general search to find information on the topic?
user: What are the health benefits of daily meditation?
loop number---- 2
Current run status: requires_action
Consulting Duck Duck Go...
[{'tool_call_id': 'call_XHxCOjPsXvlatbrjviwNBGIH', 'output': 'Summary. Self-inquiry and related styles of meditation can help you "know yourself.". This can be a starting point for making other positive changes. 5. Lengthens attention span. Focused ...\nBenefits of meditation includes physical changes to the brain and improved cognitive functioning, such as (Shapiro, 2020): Slowing of brain aging (particularly thinning of the prefrontal cortex) Improved attention. Increased innovation. Better problem-solving.\nAlthough the practice of meditation

{'completed': True}

# Make Sure You Delete Resources

Make sure you delete all the resources you created!

This function will help you do so!

In [93]:
file_deletion_status = client.beta.assistants.files.delete(
  assistant_id=assistant.id,
  file_id=file_reference.id
)